Notebook 2

# Set up

same as first notebook

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
import io
import math
import ast
from functools import partial 

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Partition6467LinkData.csv to Partition6467LinkData.csv
Saving Partition6467ProbePoints.csv to Partition6467ProbePoints.csv


In [3]:
probeCols = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading']
matchCols = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading', 
             'linkPVID', 'direction', 'distFromRef', 'distFromLink']
linkCols = ['linkPVID', 'refNodeID', 'nrefNodeID', 'length', 'functionalClass', 'directionOfTravel', 'speedCategory', 
            'fromRefSpeedLimit', 'toRefSpeedLimit', 'fromRefNumLanes', 'toRefNumLanes', 'multiDigitized', 'urban', 
            'timeZone', 'shapeInfo', 'curvatureInfo', 'slopeInfo']

In [4]:
probePoints = pd.read_csv(io.BytesIO(uploaded['Partition6467ProbePoints.csv']), ',', header=None, names=probeCols)
linkData = pd.read_csv(io.BytesIO(uploaded['Partition6467LinkData.csv']), ',', header=None, names=linkCols)

# Parse

same as first notebook, and includes creation of `incident` column

In [5]:
def stof(x):
  if x is not '':
    return float(x)
  else:
    return np.NaN

def parseInfo(x):
  if x is not None and x is not np.NaN:
    a = x.split('|')
    z = []
    for y in a:
      z.append(tuple(map(stof, y.split('/'))))
    return z

In [6]:
linkData['shapeInfo'] = linkData['shapeInfo'].apply(parseInfo)
linkData['curvatureInfo'] = linkData['curvatureInfo'].apply(parseInfo)
linkData['slopeInfo'] = linkData['slopeInfo'].apply(parseInfo)

In [ ]:
# For each road link, add a column with a list of links to which it is incident
i = 0
linkData['incident'] = None
for l in linkData.itertuples():
  if i % 500 == 0:
    print(i, end=', ')
  if i % 10000 == 0:
    print()
  i += 1

  refNode = l[2]
  nonRefNode = l[3]
  ilinks = linkData[(linkData.refNodeID == refNode) | 
                    (linkData.refNodeID == nonRefNode) | 
                    (linkData.nrefNodeID == refNode) | 
                    (linkData.nrefNodeID == nonRefNode)]
  linkData.at[l[0], 'incident'] = ilinks['linkPVID'].tolist()

# Approach #2 (revisions)

## Unchanged cells

In [50]:
# create the output dataframe
matchedPoints = pd.DataFrame(columns=matchCols)

In [51]:
print(matchedPoints)

Empty DataFrame
Columns: [sampleID, dateTime, sourceCode, latitude, longitude, altitude, speed, heading, linkPVID, direction, distFromRef, distFromLink]
Index: []


In [9]:
# Find all unique SampleIDs (trajectories)
samples = probePoints['sampleID']
trajectories = set(())
for s in samples:
  trajectories.add(s)

`minDistance`: same functionality as `dist_to_link`, but uncessary operations and variables are removed

In [10]:
# distance from point to line
# geeksforgeeks.org/minimum-distance-from-a-point-to-the-line-segment-using-vectors
def minDistance(A, B, E):
    if ((B[0]-A[0]) * (E[0]-B[0]) + (B[1]-A[1]) * (E[1]-B[1]) > 0) or ((B[0]-A[0]) * (E[0]-A[0]) + (B[1]-A[1]) * (E[1]-A[1]) < 0):
      return -1
    mod = np.sqrt((B[0]-A[0])*(B[0]-A[0]) + (B[1]-A[1])*(B[1]-A[1]))
    return 0 if mod == 0 else abs((B[0]-A[0]) * (E[1]-A[1]) - (B[1]-A[1]) * (E[0]-A[0])) / mod

`distance`: same functionality as `dist_to_point`, but edited in an attempt to speed it up

In [25]:
# calculate distance between two points
def distance(A, B):
  return math.sqrt((B[0]-A[0])*(B[0]-A[0]) + (B[1]-A[1])*(B[1]-A[1]))

sort `probePoints` first so that it does not have to be sorted in the main algorithm

In [12]:
# sort this new df by the datetime column
probePoints.sort_values('dateTime', inplace=True)

## Approach #2 (revision 1: slower, buggy, DO NOT RUN)



These functions enable the usage of the pandas dataframe `apply` function, which has been described by multiple sources as a way to make iterative algorithms more efficient; however, this ended up being the slower technique. `partialApplyMinDist` was tried once and then bypassed by a `lambda` instead. Neither ended up being faster than the other.

In [ ]:
# curried functions for new (slower) method
def applyMinDist(p, x):
  ref = x[14][0]
  nonref = x[14][-1]
  return minDistance(ref, nonref, p)

def partialApplyMinDist(p):
  return partial(applyMinDist, p)

The main algorithm, except the `apply` dataframe function is used in conjunction with the partially applied function above. Because of how `apply` works and the fact that `minDistance` takes two parameters, it was necessary to create the secondary, partially applied function.

Once it was determined that `partialApplyMinDist` didn't improve runtime, it was postulated that this incurred too much overhead, so it was replaced with a simple `lambda` function. This did not change the runtime.

In [ ]:
# iterate over each trajectory
i = 1
for t in trajectories:
  print("{}".format(t), end=', ')
  if i % 100 == 0:
    print()
  i += 1

  # get the slice of the df that is relevant to this trajectory (sampleID)
  singleProbe = probePoints[probePoints.sampleID == t]

  ituples = singleProbe.itertuples()
  nearestPoint = None
  nearestDist = None
  
  # for the first point, search all links for nearest
  firstPoint = next(ituples)
  p = (firstPoint[4], firstPoint[5])
  distances = linkData.apply(lambda x: applyMinDist(p, x), axis=1)
  nearestPoint = linkData.iloc[distances.idxmin()]
  nearestDist = distances.min()

  # create and append the first row for this trajectory
  row = {
    'sampleID': firstPoint[1],
    'dateTime': firstPoint[2],
    'sourceCode': firstPoint[3],
    'latitude': firstPoint[4],
    'longitude': firstPoint[5],
    'altitude': firstPoint[6],
    'speed': firstPoint[7],
    'heading': firstPoint[8],
    'linkPVID': nearestPoint[0],
    'direction': nearestPoint[5],
    'distFromRef': distance((firstPoint[4], firstPoint[5]), nearestPoint[14][0]),
    'distFromLink': nearestDist
  }
  matchedPoints.append(row, ignore_index=True)

  # for all subsequent points, search all links that
  # are incident to the previous for the nearst
  for point in ituples:
    incidentLinks = nearestPoint[17]
    for il in incidentLinks: # il is a linkPVID incident to nearestPoint
      l = linkData.loc[linkData['linkPVID'] == il].iloc[0]
      ref = l[14][0]
      nonref = l[14][-1]
      p = (point[4], point[5])
      d = minDistance(ref, nonref, p)
      if d < nearestDist:
        nearestPoint = l
        nearestDist = d

    # create and append the next row for this trajectory
    row = {
      'sampleID': point[1],
      'dateTime': point[2],
      'sourceCode': point[3],
      'latitude': point[4],
      'longitude': point[5],
      'altitude': point[6],
      'speed': point[7],
      'heading': point[8],
      'linkPVID': nearestPoint[0],
      'direction': nearestPoint[5],
      'distFromRef': distance((point[4], point[5]), nearestPoint[14][0]),
      'distFromLink': nearestDist
    }
    matchedPoints.append(row, ignore_index=True)

## Approach #2 (final revision, bug fixes)

The main algorithm, reverted away from the pandas `apply` function and opted for regular iteration. Although the fastest of all the revisions, it is not fast enough to be convenient. Some tweaks and bug fixes have been made here which differentiate it from previous versions.

In [52]:
# iterate over each trajectory
i = 1
for t in trajectories:
  print(t, end=', ')
  if i % 100 == 0:
    print()
  i += 1
  if i == 1000:
    break

  # get the slice of the df that is relevant to this trajectory (sampleID)
  singleProbe = probePoints[probePoints.sampleID == t]

  ituples = singleProbe.itertuples()
  nearestPoint = None
  nearestDist = None
  
  # for the first point, search all links for nearest
  firstPoint = next(ituples)
  p = (firstPoint[4], firstPoint[5])
  for l in linkData.itertuples(index=False):
    ref = l[14][0]
    nonref = l[14][-1]
    d = minDistance(ref, nonref, p)
    if nearestPoint is None or nearestDist is None or nearestDist < 0 or (d > 0 and d < nearestDist):
      nearestPoint = l
      nearestDist = d

  # create and append the first row for this trajectory
  row = {
    'sampleID': firstPoint[1],
    'dateTime': firstPoint[2],
    'sourceCode': firstPoint[3],
    'latitude': firstPoint[4],
    'longitude': firstPoint[5],
    'altitude': firstPoint[6],
    'speed': firstPoint[7],
    'heading': firstPoint[8],
    'linkPVID': nearestPoint[0],
    'direction': nearestPoint[5],
    'distFromRef': distance((firstPoint[4], firstPoint[5]), nearestPoint[14][0]),
    'distFromLink': nearestDist
  }
  matchedPoints = matchedPoints.append(row, ignore_index=True)

  # for all subsequent points, search all links that
  # are incident to the previous for the nearst
  for point in ituples:
    incidentLinks = nearestPoint[17]
    nearestPoint = None
    nearestDist = None
    for il in incidentLinks: # il is a linkPVID incident to nearestPoint
      l = linkData.loc[linkData['linkPVID'] == il].iloc[0]
      ref = l[14][0]
      nonref = l[14][-1]
      p = (point[4], point[5])
      d = minDistance(ref, nonref, p)
      if nearestPoint is None or nearestDist is None or nearestDist < 0 or (d > 0 and d < nearestDist):
        nearestPoint = l
        nearestDist = d

    # create and append the next row for this trajectory
    row = {
      'sampleID': point[1],
      'dateTime': point[2],
      'sourceCode': point[3],
      'latitude': point[4],
      'longitude': point[5],
      'altitude': point[6],
      'speed': point[7],
      'heading': point[8],
      'linkPVID': nearestPoint[0],
      'direction': nearestPoint[5],
      'distFromRef': distance((point[4], point[5]), nearestPoint[14][0]),
      'distFromLink': nearestDist
    }
    matchedPoints = matchedPoints.append(row, ignore_index=True)
    

4456449, 1179651, 2359299, 3538948, 3538950, 4587527, 4587528, 4718597, 1310730, 5505027, 5505033, 2490382, 3021986, 4980752, 1703955, 1703956, 1703957, 1703958, 2490387, 3801111, 3801112, 3334607, 4980758, 4980766, 5601095, 4980771, 2883620, 5767206, 5418736, 2621482, 4849706, 3801132, 4980781, 5242926, 2097200, 2097201, 3407926, 1310775, 2359352, 5767226, 5505090, 5767236, 4798517, 3276871, 3407950, 5242959, 4194386, 4718674, 4980818, 3276888, 3539032, 4194394, 3801183, 2883682, 3276898, 4194403, 4980838, 917608, 3145838, 1572975, 5111925, 1589170, 4587641, 1979944, 4456572, 1589171, 1572991, 917632, 917633, 917634, 917635, 917636, 1572992, 1572993, 1572994, 1572995, 1589172, 2752643, 3801219, 3751455, 5111952, 5505169, 5158242, 2490523, 3408028, 1310878, 5243043, 4980908, 5243052, 3014832, 4272496, 5767356, 3932352, 3670209, 4980931, 4194502, 2500991, 4980936, 3670218, 4642219, 
4849873, 3277012, 3277013, 3277014, 3277015, 3277016, 3277017, 3277018, 2883803, 3539159, 4849885, 484988

In [53]:
# export new link data as CSV
matchedPoints.to_csv('Partition6467MatchedPoints.csv')
files.download('Partition6467MatchedPoints.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
click = 1